In [1]:
from modelzipper.tutils import *
import sys
import os
sys.path.append("/data/zecheng/acl2025/MyRLHF/reetrievalheaddetect")
from retrieval_head_detection import SentenceSampler
import datasets
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

model = AutoModelForCausalLM.from_pretrained("/data/zecheng/hf_models/Meta-Llama-3.1-8B-Instruct", device_map='balanced_low_0', torch_dtype=torch.bfloat16, attn_implementation="eager")
tokenizer = AutoTokenizer.from_pretrained("/data/zecheng/hf_models/Meta-Llama-3.1-8B-Instruct")
data = auto_read_data("/data/zecheng/acl2025/MyRLHF/reetrievalheaddetect/haystack_for_detect/reasoning_needle.jsonl")

# needle
needles_and_stacks = [json.loads(l) for l in open(f"/data/zecheng/acl2025/MyRLHF/reetrievalheaddetect/haystack_for_detect/reasoning_needle.jsonl")]
needle_list = [l["needle"] for l in needles_and_stacks]
retrieval_question_list = [l["question"] for l in needles_and_stacks]
evidence_list = [l["real_needle"] for l in needles_and_stacks]
golden_answer_list = [l["golden_answer"] for l in needles_and_stacks]
tags = [l["tag"] for l in needles_and_stacks]

selected_idx = 0
needle = [tokenizer(i)['input_ids'] for i in needle_list[selected_idx]]
evidence = [tokenizer(i)['input_ids'] for i in evidence_list[selected_idx]]
question = retrieval_question_list[selected_idx]
answer = golden_answer_list[selected_idx]
tag = tags[selected_idx]


# 初始化采样器
haystack = datasets.load_dataset("/data/data/zecheng/data/pg19-test", split="test")
noise_sampler_test = SentenceSampler(haystack, tokenizer=tokenizer, shuffle=False, random_seed=None)
background_text = noise_sampler_test.get_sample(15500)
disturb_tok_needles = [i for i in needle if i not in evidence]
disturb_pos = np.random.choice(len(background_text)+1, len(disturb_tok_needles))
depth_percent = [0.1, 0.2, 0.4]

updated_sample = [[] for _ in range(len(background_text) + 1)]
real_pos = [int(len(background_text) * i) for i in depth_percent]
for fact, pos in zip(evidence, real_pos):  # insert real needle
    updated_sample[pos].append(fact)
for fact, pos in zip(disturb_tok_needles, disturb_pos):  # insert disturb needle
    updated_sample[pos].append(fact)
for i, s in enumerate(background_text):  # insert irrevelent needle
    updated_sample[i].append(s)

flat = [i for s in updated_sample for i in s]
tokens = [i for s in flat for i in s]

new_context = tokenizer.decode(tokens)
input_context = new_context + f"\nQuestion: {question}\nAnswer:"
inp = tokenizer.apply_chat_template([{ "role": "user", "content": input_context}], tokenize=True, return_tensors='pt').to(model.device)

print(inp.shape)

# inp = tokenizer("hello, who are you?", return_tensors="pt").input_ids.to('cuda:1')

ModelZipper is ready for launch🚀 | Current Version🦄 >>> 0.2.7 <<< | AOE Time🕒 2024-12-16 17:53:36


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

begin to read data from /data/zecheng/acl2025/MyRLHF/reetrievalheaddetect/haystack_for_detect/reasoning_needle.jsonl | file size: 7.03 KB | file type: jsonl
torch.Size([1, 15767])


In [2]:
from baukit import Trace
with torch.no_grad(), Trace(model.model.layers[8], "self_attn") as ret:
    _ = model(inp, output_attentions=True)
    representation = ret.output

OutOfMemoryError: CUDA out of memory. Tried to allocate 29.64 GiB. GPU 1 has a total capacity of 94.99 GiB of which 13.55 GiB is free. Including non-PyTorch memory, this process has 81.43 GiB memory in use. Of the allocated memory 36.43 GiB is allocated by PyTorch, and 44.57 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [16]:
print(representation[0].shape)

print(representation[1].shape)

torch.Size([1, 7, 4096])
torch.Size([1, 32, 7, 7])


In [9]:
model.model.layers[0]

LlamaDecoderLayer(
  (self_attn): LlamaSdpaAttention(
    (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
    (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
    (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
    (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
    (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
)